# <center>Лабораторная работа № 17
## <center> Прогнозирование задержек вылетов

В этой лабоарторной работе вам предлагается взять участие в [соревновании](https://www.kaggle.com/c/flight-delays-2017) на Kaggle Inclass.
    
Предскажите задержки вылета в 15 и более минут. Известно время вылета, авиакомпания-перевозчик, место вылета и назначения, а также расстояние между ними. Прогнозируемый признак – бинарный, целевая метрика – ROC AUC. 
    
Это тот тип задач, в которой XGBoost проявит себя наилучшем образом (так же не забывайте о Catboost).

![image](https://raw.githubusercontent.com/soolstafir/Machine-learning-course/master/img/xgboost_meme.jpg)

In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score

In [3]:
train_csv = 'https://raw.githubusercontent.com/soolstafir/Machine-learning-course/master/Labs/ml_lab-17_flight-delays-kaggle/flight_delays_train.csv'
test_csv = 'https://raw.githubusercontent.com/soolstafir/Machine-learning-course/master/Labs/ml_lab-17_flight-delays-kaggle/flight_delays_test.csv'

train = pd.read_csv(train_csv)
test = pd.read_csv(test_csv)

**Описание набора данных:**

flight_delays_train.csv – обучающая выборка

flight_delays_test.csv – тестовая выборка

sample_submission.csv – пример файла посылки

Признаки:

- `Month` - месяц вылета
- `DayofMonth` - день месяца вылета
- `DayOfWeek` - день недели вылета
- `DepTime` - время отправления
- `UniqueCarrier` - код перевозчика
- `Origin` - место вылета
- `Dest` - место назначения
- `Distance` - расстояние между аэропортами вылета и прилета
- `dep_delayed_15min` - просрочка вылета на 15 и более минут (целевой признак)

In [8]:
train.head()

Month DayofMonth DayOfWeek  DepTime UniqueCarrier Origin Dest  Distance  \
0   c-8       c-21       c-7     1934            AA    ATL  DFW       732   
1   c-4       c-20       c-3     1548            US    PIT  MCO       834   
2   c-9        c-2       c-5     1422            XE    RDU  CLE       416   
3  c-11       c-25       c-6     1015            OO    DEN  MEM       872   
4  c-10        c-7       c-6     1828            WN    MDW  OMA       423   

  dep_delayed_15min  
0                 N  
1                 N  
2                 N  
3                 N  
4                 Y

In [9]:
test.head()

Month DayofMonth DayOfWeek  DepTime UniqueCarrier Origin Dest  Distance
0   c-7       c-25       c-3      615            YV    MRY  PHX       598
1   c-4       c-17       c-2      739            WN    LAS  HOU      1235
2  c-12        c-2       c-7      651            MQ    GSP  ORD       577
3   c-3       c-25       c-7     1614            WN    BWI  MHT       377
4   c-6        c-6       c-3     1505            UA    ORD  STL       258

В целом, следует предсказать задержку вылета более 15 минут по времени вылета самолета, коду авиакомпании-перевозчика, месту вылета и прилета и расстоянию между аэропортами вылета и прилета. 

В качестве простейшего бенчмарка можно взять логистическую регрессию и два признака, например,  `DepTime` и `Distance` (их проще всего взять). У такой модели результат – 0.68202 на LB. 

In [10]:
X_train, y_train = train[['Distance', 'DepTime']].values, train['dep_delayed_15min'].map({'Y': 1, 'N': 0}).values
X_test = test[['Distance', 'DepTime']].values

X_train_part, X_valid, y_train_part, y_valid = train_test_split(X_train, 
                                                                y_train, 
                                                                test_size=0.3, 
                                                                random_state=17)

In [11]:
logit = LogisticRegression(random_state=17)

logit.fit(X_train_part, y_train_part)
logit_valid_pred = logit.predict_proba(X_valid)[:, 1]

roc_auc_score(y_valid, logit_valid_pred)

0.6795697123357751

In [12]:
logit.fit(X_train, y_train)
logit_test_pred = logit.predict_proba(X_test)[:, 1]

pd.Series(logit_test_pred, 
          name='dep_delayed_15min').to_csv('logit_2feat.csv', 
                                           index_label='id', 
                                           header=True)

<span style="color:red; font-size:2em;">Задание</span>

**Предскажите задержку вылета на более, чем 15 минут (`dep_delayed_15min`) по следующим признакам:**

- `Month` - месяц вылета
- `DayofMonth` - день месяца вылета
- `DayOfWeek` - день недели вылета
- `DepTime` - время отправления
- `UniqueCarrier` - код авиакомпании-перевозчика
- `Origin` - место вылета
- `Dest` - место назначения
- `Distance` - расстояние между аэропортами вылета и прилета

Рекомендации к построению бенчмарка по примеру [соревнования](https://www.kaggle.com/c/flight-delays-2017):
- Признаки `Distance` и  `DepTime` возьмем без изменений.
- Создадим признак "маршрут" из исходных `Origin` и `Dest`.
- К признакам `Month`, `DayofMonth`, `DayOfWeek`, `UniqueCarrier` и "маршрут" применим OHE-преобразование (`LabelBinarizer`).
- Выделим отложенную выборку.
- Обучим логистическую регрессию и градиентный бустинг (xgboost).
- Настроим гиперпараметры бустинга на кросс-валидации; сначала те, что отвечают за сложность модели, затем число деревьев зафиксируем равным 500 и настроим шаг градиентного спуска.
- С помощью `cross_val_predict` сделаем прогнозы обеих моделей на кросс-валидации (именно предсказанные вероятности).
- Настроим линейную смесь ответов логистической регрессии и градиентного бустинга вида $w_1 * p_{logit} + (1 - w_1) * p_{xgb}$, где $p_{logit}$ – предсказанные логистической регрессией вероятности класса 1, $p_{xgb}$ – аналогично. Вес $w_1$ можноподобрать вручную. 
- В качестве ответа для тестовой выборки можно взять аналогичную комбинацию ответов двух моделей, но уже обученных на всей обучающей выборке.

In [13]:
X_train.shape

(100000, 2)

In [14]:
test.shape

(100000, 8)

In [15]:
train.shape

(100000, 9)

In [16]:
X_train_part.shape

(70000, 2)

In [19]:
X_valid.shape

(30000, 2)

In [21]:
train.head()

Month DayofMonth DayOfWeek  DepTime UniqueCarrier Origin Dest  Distance  \
0   c-8       c-21       c-7     1934            AA    ATL  DFW       732   
1   c-4       c-20       c-3     1548            US    PIT  MCO       834   
2   c-9        c-2       c-5     1422            XE    RDU  CLE       416   
3  c-11       c-25       c-6     1015            OO    DEN  MEM       872   
4  c-10        c-7       c-6     1828            WN    MDW  OMA       423   

  dep_delayed_15min  
0                 N  
1                 N  
2                 N  
3                 N  
4                 Y

In [23]:
test.head()

Month DayofMonth DayOfWeek  DepTime UniqueCarrier Origin Dest  Distance
0   c-7       c-25       c-3      615            YV    MRY  PHX       598
1   c-4       c-17       c-2      739            WN    LAS  HOU      1235
2  c-12        c-2       c-7      651            MQ    GSP  ORD       577
3   c-3       c-25       c-7     1614            WN    BWI  MHT       377
4   c-6        c-6       c-3     1505            UA    ORD  STL       258